___
# Covalent Alchemist tasks - 1inch V3 Aggregator
### Question: Find the largest \$USD swap via 1inch V3
___

Not sure, let's find out.

With my hacky python knoweldge, unclear understanding of the ethereum blockchain and a will to stubbornness, i've barely muddled together half a solution for only the first question... As a beginner, I really appreciate the proivided low-hanging–fruit questions so thanks, though I was unable to complete it.

#### Info grabbed from Etherscan

- [Contract Creation](https://etherscan.io/tx/0x9825b9f083ba43112afd66dbcfbbf770e24bc21ff4128ce24ddbda171d79ae2f) occurs at block 11511393
- [Some block (2021-05-31)](https://etherscan.io/block/12538697) 12538697 (approximation for `latest` for the sake of arithmetic below)

#### Covalent API Limitations
``` json
{
    'data': None,
     'error': True,
     'error_message': 'Block ranges greater than 1 million'
                         'not currently supported (got 11511394)',
     'error_code': 501
 }
 ```
 
$\text{current-ish} - \text{contract creation} = 1,027,304$ blocks 🙄

1. Sort out params
2. Grab relevant block heights
3. Grab event log of contract
4. Pick out transaction hashes
5. Pick out swaps
6. Grab largest

In [ ]:
# --- 1. Sort out Parameters 

import requests
import numpy as np
import seaborn as sns
import datetime as dt

eth_chain = 1
bsc_chain = 56
inch_addr = "0x111111111117dc0aa78b770fa6a738034120c302"
start_block = 11511393 # 1inch V3 contract creation block
today_str = dt.datetime.today().strftime("%Y-%m-%d")

# --- Grab auth c_key from file

with open('auth.txt') as f:
    auth = (f.readlines()[0].strip(), "")

# --- base url

base_url = "https://api.covalenthq.com/v1"
contract_url = "/{chain}/events/address/{addr}/"
contract_params = "\
?starting-block={start_block}\
&ending-block={end_block}\
&page-number={pg_no}\
&quote-currency=usd\
"
height_url ="/{chain}/block_v2/{start_date}/{end_date}/"
tx_url = "/{chain}/transaction_v2/{tx_hash}/"

In [ ]:
# --- Latest (enough) block heights function

def get_latest_height(url, chain):
    r_height = requests.get(
                            url.format(chain=chain, start_date=today_str, end_date="latest"),
                            auth=auth
                           ).json()
    heights = [s["height"] for s in r_height["data"]["items"]]
    return max(heights)

# --- get contract event log function 
def get_events(url, chain, addr, auth, start_block, end_block,  events=None):
    events = [] if events is None else events
    
    # Break up range if greater than 1m
    n_blocks = int(end_block) - int(start_block)
    if n_blocks > 1e6:
        n_intervals = n_blocks // 1e6
        intervals = [[start_block, start_block + 1e6],
                     [start_block + 1e6, end_block]]
    else:
        intervals = [[start_block, end_block]]
        
    healthy = True
    page = 3
    
    for i in intervals:
        i_start, i_end = int(i[0]), int(i[1])
        while healthy:
            print("Page ", page)
            get_contract_url = url.format(
                                    chain=chain, addr=addr,
                                    start_block=i_start, end_block=i_end,
                                    pg_no=page
            )
            event_json = requests.get(get_contract_url, auth=auth).json()
            if event_json["error"]:
                print(event_json["error_code"])
                page += 1
            else:
                new_events = event_json["data"]["items"]
                print("n. new events: ", len(new_events))
                events.extend(new_events)
                print("Length of events: ", len(events))
                page += 1
                
            if page == 5: # keeping it short for now. Remove to go through all the pages
                return events
            
def get_tx(tx_hash, chain):
    url = base_url + tx_url
    url = url.format(chain=chain, tx_hash=tx_hash)
    tx_json = requests.get(url, auth=auth).json()
    content = tx_json["data"]["items"]
    kind = content[""]
    return tx_json

In [ ]:
# --- Grab 1inch Event log

get_height_url = base_url + height_url
end_block = get_latest_height(get_height_url, eth_chain)

events_url = base_url + contract_url + contract_params
inch_events = get_events(url=events_url, chain=eth_chain,
                    addr=inch_addr, auth=auth,
                    start_block=start_block, end_block=end_block)

In [ ]:
# --- Check out example transaction here:

inch_events[165]

In [ ]:
# --- Inspect one tx as an example

tx = "0xecea96f102acc3c4ae87d54c9eb906109e5c86b961eb5a590fc24718c80ceba0"
tx = get_tx(tx, chain=eth_chain)

Now I need to discern whether the event was a swap or a claim token transfer, similar. I can't figure out how to do this from the info above, but putting the `tx_hash` through covalent's "get transaction" API spells out what's going on in plain english. The thought of shoving all these transactions thorugh re pains me. There must be a faster way.. but for the sake of a solution that technically works, I would press forward, but i'm out of time. I wish I could have given this task more time